# **CREACION DE UNA BASE DE DATOS ANALITICA**

### **EVIDENCIA DE APRENDISAJE 1 y 2**

**INTEGRANTES:**

**OSCAR LUIS MARQUEZ ARRIETA**

**NATALIA JADITH CASTRO OSPINO**

**PREICA2502B020061**

## **PROBLEMATICA**

La empresa de fluido electrico EnergyCol especializada en el servicio de energia electrica a gran parte de las regiones de colombia, tanto residenciales como comerciales. Esta presentando problemas con el procesamineto de los datos de todos los clientes debido al alto tamaño de informacion que se tienen de todos los clientes.

Debido al a esta problematica, se implementara una base de datos bien estructurada y organiza, que permita implementar toda la informacion que se requiera de los usuarios. Esto le brindara a la empresa una mejor organizacion y las relaciones entre el tamaño del edificio, la ocupación, la ubicación geográfica y los costos de energía.

## **Dataset**

**Fuente:** Kaggle: Residential and Commercial Energy Cost Dataset

**Enlace:** https://www.kaggle.com/datasets/andreylss/residential-and-commercial-energy-cost-dataset

# **VARIABLES RELEVANTES**

**id_cliente:** Identificador único para cada cliente

**tipo_cliente:** Tipo de propiedad (residencial o comercial)

**costo_energía_brl:** Costo mensual de energía en moneda local

**tipo_cliente:** Tipo de propiedad (residencial o comercial)

**regiones:** Región geográfica (Norte, Noreste, Medio Oeste, Sudeste, Sur)

**Este Dataset lo escogimos porque se complementa muy bien con los datos que tenemos de todos los usuarios de nuestar empresa. Estos nos facilita hacer una muy buena migracion a esta base datos muy organizada y funcional para todo lo que se necesita respecto a los datos de nuestros usuarios.**










# **Modelo Entidad-Relación (ERD)**

![Diagrama Entidad-Relación](./ERD_ENERGYCOL.png)

Este diagrama representa un modelo de base de datos relacional para gestionar el consumo de energía de la empres EnergyCol.

**REGION**


Contiene información sobre las regiones geográficas.


Campos:


- id_region: Identificador único de la región (PK).

- nombre_region: Nombre de la región.

- tarifa_promedio_brl: Tarifa promedio de energía en reales brasileños (BRL).



**CLIENTE**


Almacena los datos de los clientes.


Campos:


- id_cliente: Identificador único del cliente (PK).

- nombre_cliente: Nombre del cliente.

- tipo_cliente: Tipo de cliente (residencial, comercial, industrial, etc.).

- direccion: Dirección del cliente.

- id_region: Llave foránea que conecta con la tabla REGION (FK).




**CONSUMO_ENERGIA**


Registra el consumo mensual de energía de cada cliente.


Campos:


- id_consumo: Identificador único del registro de consumo (PK).

- id_cliente: Llave foránea que conecta con CLIENTE (FK).

- mes y anio: Período de consumo.

- consumo_kwh: Energía consumida en kilovatios-hora.

- costo_energia_brl: Costo correspondiente al consumo en reales.



**Relaciones:**


Una región puede tener muchos clientes.

Un cliente puede tener muchos registros de consumo de energía.



# **Creacion BD + Carga de datos**

In [0]:
!pip install kagglehub[pandas-datasets]>=0.3.8

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## **Librerias implementadas**

In [0]:

import kagglehub
import pandas as pd
import os
import zipfile

## **Descargamos, extraemos y leemos el dataset de Kaggle**

In [0]:

def download_dataset_zip(url = ""):
        print("Descargando dataset desde Kaggle...")
        dataset_path = kagglehub.dataset_download(url)
        print("Ruta al dataset:", dataset_path)
        return dataset_path
    
def extract_zip_files(dataset_path):
        zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
        if zip_files:
            zip_file = os.path.join(dataset_path, zip_files[0])
            extract_dir = os.path.join(dataset_path, "extracted")
            os.makedirs(extract_dir, exist_ok=True)
            print(f"Extrayendo {zip_file} en {extract_dir}...")
            with zipfile.ZipFile(zip_file, "r") as z:
                z.extractall(extract_dir)
            return extract_dir
        else:
            # Si no se encuentra un ZIP, se verifica si existen archivos CSV en la ruta
            csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
            if csv_files:
                print("No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.")
                return dataset_path
            else:
                raise FileNotFoundError("No se encontró ningún archivo .zip ni archivos .csv en la ruta del dataset")

def create_csv(csv_dir):
        #os.makedirs('src/static/csv', exist_ok=True)
        csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No se encontraron archivos CSV en el directorio extraído")

        for file in csv_files:
            file_path = os.path.join(csv_dir, file)
            print(f"Leyendo {file_path}...")
            try:
                df = pd.read_csv(file_path, encoding="latin1")
            except Exception as e:
                print(f"Error al leer {file}: {e}")
                continue
            print(f"Creando/actualizando ")
        print("cvs creado correctamente en ")
        return df

## **Descargamos Dataset**

In [0]:
df = pd.DataFrame()
dataset = download_dataset_zip("andreylss/residential-and-commercial-energy-cost-dataset") 
csv_dir =extract_zip_files(dataset)
df = create_csv(csv_dir)


Descargando dataset desde Kaggle...


100%|██████████| 38.3k/38.3k [00:00<00:00, 6.48MB/s]

Extracting files...
Ruta al dataset: /home/spark-a9f2599d-a7de-48b1-90a8-ca/.cache/kagglehub/datasets/andreylss/residential-and-commercial-energy-cost-dataset/versions/1
No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.
Leyendo /home/spark-a9f2599d-a7de-48b1-90a8-ca/.cache/kagglehub/datasets/andreylss/residential-and-commercial-energy-cost-dataset/versions/1/energy_consumption.csv...
Creando/actualizando 
cvs creado correctamente en 


## **Verificamos la carga de los datos cargados de Kaggle**

In [0]:
df.head(3)

,customer_id,customer_type,regions,building_size_m2,occupants,energy_cost_brl
0,CUSTOMER_0001,residential,Northeast,24,2,64.51
1,CUSTOMER_0002,commercial,Midwest,24,1,55.26
2,CUSTOMER_0003,commercial,Southeast,24,1,74.54


## **Convertir Dataframe de pandas a Spark**

In [0]:

spark_df = spark.createDataFrame(df)

## **Creación la Tabla e Inserto de Datos**

In [0]:
spark_df.write.mode("overwrite").saveAsTable("tbl_clientes_al_dia")

## **Ver tabla creada**

In [0]:
%sql
SELECT
  *
FROM
  tbl_clientes_al_dia
LIMIT 5;

customer_id,customer_type,regions,building_size_m2,occupants,energy_cost_brl
CUSTOMER_0001,residential,Northeast,24,2,64.51
CUSTOMER_0002,commercial,Midwest,24,1,55.26
CUSTOMER_0003,commercial,Southeast,24,1,74.54
CUSTOMER_0004,residential,Northeast,45,4,147.06
CUSTOMER_0005,residential,Southeast,45,4,143.06


## **Cosultas SQL**

## **1. Conteo de registros**

In [0]:
%sql
SELECT
  COUNT(*)
FROM
  tbl_clientes_al_dia

COUNT(*)
5000


El conteo nos arroja que la tabla tbl_**clientes_al_dia** cuenta con 5000 registros en total realizados.

## **2. Nombre y tipos de columnas**

In [0]:
%sql
DESCRIBE TABLE tbl_clientes_al_dia

col_name,data_type,comment
customer_id,string,null
customer_type,string,null
regions,string,null
building_size_m2,bigint,null
occupants,bigint,null
energy_cost_brl,double,null


Esta cosulta nos devuelve la estructura de la tabala **tbl_clientes_al_dia**, nos muestra el nombre de la columna y el tipo de dato de cada uno de los datos.

## **3. Cosulta con filtro**

In [0]:
%sql
SELECT
  *
FROM
  tbl_clientes_al_dia
WHERE
  regions = 'Northeast'
LIMIT (10);

customer_id,customer_type,regions,building_size_m2,occupants,energy_cost_brl
CUSTOMER_0001,residential,Northeast,24,2,64.51
CUSTOMER_0004,residential,Northeast,45,4,147.06
CUSTOMER_0009,residential,Northeast,45,2,70.03
CUSTOMER_0011,residential,Northeast,52,3,65.47
CUSTOMER_0015,residential,Northeast,45,1,95.75
CUSTOMER_0016,residential,Northeast,17,1,90.83
CUSTOMER_0018,residential,Northeast,24,1,84.0
CUSTOMER_0027,residential,Northeast,77,4,112.98
CUSTOMER_0031,residential,Northeast,17,1,55.69
CUSTOMER_0035,commercial,Northeast,45,2,73.36


Aqui hacemos una cosulta a la tabla **tbl_clientes_al_dia** en la que hacemos un filtro en donde le mandamos que nos muestre a los clientes que estan en la region Nor oeste. Esta nos arroja muy claramente los clientes que se encuentran en esa region, nos muestra las filas y columnas de esta tabla.

In [0]:
%sql
SELECT
  COUNT(*)
FROM
  tbl_clientes_al_dia
WHERE
  regions = 'Northeast'

COUNT(*)
1266


Aqui quise realizar un conteo de cuantos clientes hay en la region Noroeste, este nos muestra que en total hay 1266 clientes en esta region.

# **ACTIVIDAD 2 BIG DATA**

## **Punto 1**

**1.1  Describe las entidades y campos clave del dataset (tipos de datos, llaves, nulabilidad).**


**Region**
| Campo                   | Tipo de dato     | Llave | Nulabilidad | Descripción                                               |
| ----------------------- | ---------------- | ----- | ----------- | --------------------------------------------------------- |
| **id_region**           | INT              | PK    | NOT NULL    | Identificador único de la región.                         |
| **nombre_region**       | VARCHAR | —     | NULL        | Nombre descriptivo de la región.                          |
| **tarifa_promedio_brl** | DECIMAL(10,2)    | —     | NULL        | Tarifa promedio del suministro eléctrico en reales (BRL). |




**Cliente**
| Campo              | Tipo de dato     | Llave | Nulabilidad | Descripción                                                 |
| ------------------ | ---------------- | ----- | ----------- | ----------------------------------------------------------- |
| **id_cliente**     | INT              | PK    | NOT NULL    | Identificador único del cliente.                            |
| **nombre_cliente** | VARCHAR | —     | NULL        | Nombre completo del cliente.                                |
| **tipo_cliente**   | VARCHAR | —     | NULL        | Tipo de cliente (residencial, comercial, industrial, etc.). |
| **direccion**      | VARCHAR | —     | NULL        | Dirección física del cliente.                               |
| **id_region**      | INT              | FK    | NOT NULL    | Relación con la región del cliente.                         |



**Cosumo_Energia**
| Campo                 | Tipo de dato     | Llave | Nulabilidad | Descripción                                  |
| --------------------- | ---------------- | ----- | ----------- | -------------------------------------------- |
| **id_consumo**        | INT              | PK    | NOT NULL    | Identificador único del registro de consumo. |
| **id_cliente**        | INT              | FK    | NOT NULL    | Cliente asociado al consumo.                 |
| **mes**               | VARCHAR | —     | NULL        | Mes del consumo (ej. “enero”).               |
| **anio**              | INT              | —     | NULL        | Año del registro de consumo.                 |
| **consumo_kwh**       | DECIMAL(10,2)    | —     | NULL        | Cantidad de energía consumida en kWh.        |
| **costo_energia_brl** | DECIMAL(10,2)    | —     | NULL        | Costo total del consumo mensual.             |



**1.2 Propón un DDL (Spark SQL) o una StructType (PySpark) que represente el esquema.**

In [0]:
%sql
-- Usar catálogo y esquema
USE CATALOG Energia_iudigital;
USE SCHEMA energia_schema;

-- Tabla REGION
DROP TABLE IF EXISTS region;
CREATE TABLE region (
    id_region STRING,
    nombre_region STRING
) USING DELTA;


-- Tabla CLIENTE
DROP TABLE IF EXISTS cliente;
CREATE TABLE cliente (
    customer_id STRING,
    customer_type STRING,
    building_size_m2 INT,
    occupants INT,
    id_region STRING
) USING DELTA;


-- Tabla CONSUMO_ENERGIA
DROP TABLE IF EXISTS consumo_energia;
CREATE TABLE consumo_energia (
    customer_id STRING,
    energy_cost_brl DOUBLE
) USING DELTA;


    


**1.3 Incluye un diagrama simple (Mermaid/draw.io) o tabla de diccionario de datos en una celda Markdown.**


![](mermaid-diagram-2025-11-22-154423.png)

# Punto 2.
Configuración y evidencia de la infraestructura en Databricks CE


#####Databricks Community Edition crea el clúster con configuraciones predeterminadas que no pueden ser modificadas por nosotros los usuario. A continuación se presentan las capturas que evidencian la versión del entorno y los recursos asignados automáticamente.

En esta imagen se puede observar el clúster creado automaticamente por Databrick Community Edition. El sistema nos asigna por defecto la versión de Databricks Runtime y los demas recursos sin permitir edición manual.

![](/Workspace/Users/oscar.marquezla@est.iudigital.edu.co/Oscar_Marquez_Natalia_Castro_Actividad_1/Captura de pantalla 2025-11-22 224346.png)

En esta imagen se visualizan los recursos asignados al clúster ( Runtime, tipo de clúster, autoscaling... entre otros,.). En Databricks CE estos valores no pueden modificarse y se establecen de manera predeterminada.

- ![](/Workspace/Users/oscar.marquezla@est.iudigital.edu.co/Oscar_Marquez_Natalia_Castro_Actividad_1/Captura de pantalla 2025-11-22 224546.png)

![](/Workspace/Users/oscar.marquezla@est.iudigital.edu.co/Oscar_Marquez_Natalia_Castro_Actividad_1/Captura de pantalla 2025-11-22 224608.png)

Finalmente, se observa el clúster ejecutándose. Una vez iniciado, el entorno está listo para ejecutar celdas en Python y Spark dentro de Databricks CE.

![](/Workspace/Users/oscar.marquezla@est.iudigital.edu.co/Oscar_Marquez_Natalia_Castro_Actividad_1/Captura de pantalla 2025-11-22 224902.png)

#####2.1 Versión de Databricks Runtime, tipo de clúster, núcleos/RAM, autoscaling.




Nombre: Serverless Starter Warehouse (CE default)

Estado: Running

Tipo: Serverless

Cluster size: 2X-Small

Auto stop: 10 minutos de inactividad

Scaling: Min 1 – Max 1 (Autoscaling limitado por CE)

Channel / Runtime: Current (v 2025.30)

Cluster: No permite más de 1(limitación de Community Edition)



#####2.2 Versiones de Python/Spark: spark.version, spark.sparkContext.getConf().getAll().

####2.2.1 Versión

In [0]:
import sys
print("Python version:", sys.version)

Python version: 3.12.3 (main, Aug 14 2025, 17:47:21) [GCC 13.3.0]


####2.2.2 Spark Version

In [0]:
spark.version

'4.0.0'

####2.2.3 Configuración general del SparkContext

In [0]:
# imprimir keys seleccionadas de configs
configs = None
try:
    configs = spark.conf.getAll() if hasattr(spark.conf, "getAll") and callable(getattr(spark.conf, "getAll")) else getattr(spark.conf, "getAll")
except Exception:
    # fallback (si getAll no es callable, se usa directamente el dict que ya existe)
    configs = getattr(spark.conf, "getAll", spark.conf)

keys = [
    "spark.databricks.execution.timeout",
    "spark.sql.ansi.enabled",
    "spark.sql.shuffle.partitions",
    "spark.driver.memory",
    "spark.executor.memory",
    "spark.executor.cores",
    "spark.databricks.clusterUsageTags.clusterName"
]

print("Configuraciones seleccionadas (si existen):")
for k in keys:
    print(k, "=", configs.get(k, "<no disponible>"))


Configuraciones seleccionadas (si existen):
spark.databricks.execution.timeout = 9000
spark.sql.ansi.enabled = true
spark.sql.shuffle.partitions = auto
spark.driver.memory = <no disponible>
spark.executor.memory = <no disponible>
spark.executor.cores = <no disponible>
spark.databricks.clusterUsageTags.clusterName = <no disponible>


####2.2.4 Estructura del almacenamiento (DBFS)

In [0]:
import os
print("¿DBFS está disponible?:", os.path.exists("/dbfs/"))

¿DBFS está disponible?: True


Databrick community Edition no soporta Unity Catalog ni Volumes. Por ende, el entorno usa DBFS (Databricks File System) como sistema de almacenamiento. 

# **Punto 3.**

 **3.1 Opción B (Manual/URL): descarga local y carga el archivo a DBFS (UI: Upload a /FileStore o Volumes).**

In [0]:
%fs ls /Volumes/workspace/energy/energy/energy_consumption.csv


path,name,size,modificationTime
dbfs:/Volumes/workspace/energy/energy/energy_consumption.csv,energy_consumption.csv,229907,1763857445000


**3.2 Carga en Spark: lee el archivo (CSV/JSON/Parquet) con spark.read aplicando el esquema diseñado.**

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW temp_energy
USING CSV
OPTIONS (
  path '/Volumes/workspace/energy/energy/energy_consumption.csv',
  header 'true',
  inferSchema 'true'
);


-- Ver las primeras filas
SELECT * FROM temp_energy LIMIT 10;



customer_id,customer_type,regions,building_size_m2,occupants,energy_cost_brl
CUSTOMER_0001,residential,Northeast,24,2,64.51
CUSTOMER_0002,commercial,Midwest,24,1,55.26
CUSTOMER_0003,commercial,Southeast,24,1,74.54
CUSTOMER_0004,residential,Northeast,45,4,147.06
CUSTOMER_0005,residential,Southeast,45,4,143.06
CUSTOMER_0006,residential,North,52,2,96.7
CUSTOMER_0007,residential,Midwest,17,3,52.99
CUSTOMER_0008,commercial,Southeast,45,3,102.13
CUSTOMER_0009,residential,Northeast,45,2,70.03
CUSTOMER_0010,commercial,Midwest,45,4,158.14


In [0]:
%sql
-- REGIONES únicas
INSERT INTO region
SELECT DISTINCT regions AS nombre_region,
       regions AS id_region
FROM temp_energy;


-- CLIENTES
INSERT INTO cliente
SELECT customer_id,
       customer_type,
       building_size_m2,
       occupants,
       regions AS id_region
FROM temp_energy;


-- CONSUMO_ENERGIA
INSERT INTO consumo_energia
SELECT customer_id,
       energy_cost_brl
FROM temp_energy;




num_affected_rows,num_inserted_rows
5000,5000


In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Definir el esquema basado en el CSV
schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("customer_type", StringType(), True),
    StructField("regions", StringType(), True),
    StructField("building_size_m2", IntegerType(), True),
    StructField("occupants", IntegerType(), True),
    StructField("energy_cost_brl", DoubleType(), True)
])

# Leer el CSV usando spark.read con el esquema definido
df = spark.read.csv(
    "/Volumes/workspace/energy/energy/energy_consumption.csv",  # ruta del archivo CSV
    header=True,
    schema=schema
)

# Mostrar las primeras filas
df.show(5)

+-------------+-------------+---------+----------------+---------+---------------+
|  customer_id|customer_type|  regions|building_size_m2|occupants|energy_cost_brl|
+-------------+-------------+---------+----------------+---------+---------------+
|CUSTOMER_0001|  residential|Northeast|              24|        2|          64.51|
|CUSTOMER_0002|   commercial|  Midwest|              24|        1|          55.26|
|CUSTOMER_0003|   commercial|Southeast|              24|        1|          74.54|
|CUSTOMER_0004|  residential|Northeast|              45|        4|         147.06|
|CUSTOMER_0005|  residential|Southeast|              45|        4|         143.06|
+-------------+-------------+---------+----------------+---------+---------------+
only showing top 5 rows


**3.3 Persistencia: crea una tabla (saveAsTable o CREATE TABLE USING) y muestra %sql DESCRIBE TABLE.**

In [0]:
%sql
-- crear tabla using
CREATE TABLE IF NOT EXISTS energia_consumo
USING DELTA
AS 
SELECT *
FROM csv.`/Volumes/workspace/energy/energy/energy_consumption.csv`
    


num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE TABLE energia_consumo;

col_name,data_type,comment
_c0,string,null
_c1,string,null
_c2,string,null
_c3,string,null
_c4,string,null
_c5,string,null


# **Punto 4**

**4.1 Metadatos: %sql DESCRIBE TABLE, SHOW CREATE TABLE; en Spark: df.printSchema().**

In [0]:
%sql
-- Ver columnas, tipos y nulabilidad
DESCRIBE TABLE region;

col_name,data_type,comment
id_region,string,null
nombre_region,string,null


In [0]:
%sql
-- Ver columnas, tipos y nulabilidad
DESCRIBE TABLE cliente;

col_name,data_type,comment
customer_id,string,null
customer_type,string,null
building_size_m2,int,null
occupants,int,null
id_region,string,null


In [0]:
%sql
-- Ver columnas, tipos y nulabilidad
DESCRIBE TABLE consumo_energia;

col_name,data_type,comment
customer_id,string,null
energy_cost_brl,double,null


In [0]:
%sql
-- Mostrar cómo se creó la tabla
SHOW CREATE TABLE region;

createtab_stmt
"CREATE TABLE energia_iudigital.energia_schema.region ( id_region STRING, nombre_region STRING) USING delta COLLATION 'UTF8_BINARY' TBLPROPERTIES ( 'delta.enableDeletionVectors' = 'true', 'delta.enableRowTracking' = 'true', 'delta.feature.appendOnly' = 'supported', 'delta.feature.deletionVectors' = 'supported', 'delta.feature.domainMetadata' = 'supported', 'delta.feature.invariants' = 'supported', 'delta.feature.rowTracking' = 'supported', 'delta.minReaderVersion' = '3', 'delta.minWriterVersion' = '7', 'delta.parquet.compression.codec' = 'zstd')"


In [0]:
%sql
SHOW CREATE TABLE cliente;

createtab_stmt
"CREATE TABLE energia_iudigital.energia_schema.cliente ( customer_id STRING, customer_type STRING, building_size_m2 INT, occupants INT, id_region STRING) USING delta COLLATION 'UTF8_BINARY' TBLPROPERTIES ( 'delta.enableDeletionVectors' = 'true', 'delta.enableRowTracking' = 'true', 'delta.feature.appendOnly' = 'supported', 'delta.feature.deletionVectors' = 'supported', 'delta.feature.domainMetadata' = 'supported', 'delta.feature.invariants' = 'supported', 'delta.feature.rowTracking' = 'supported', 'delta.minReaderVersion' = '3', 'delta.minWriterVersion' = '7', 'delta.parquet.compression.codec' = 'zstd')"


In [0]:
%sql
SHOW CREATE TABLE consumo_energia;

createtab_stmt
"CREATE TABLE energia_iudigital.energia_schema.consumo_energia ( customer_id STRING, energy_cost_brl DOUBLE) USING delta COLLATION 'UTF8_BINARY' TBLPROPERTIES ( 'delta.enableDeletionVectors' = 'true', 'delta.enableRowTracking' = 'true', 'delta.feature.appendOnly' = 'supported', 'delta.feature.deletionVectors' = 'supported', 'delta.feature.domainMetadata' = 'supported', 'delta.feature.invariants' = 'supported', 'delta.feature.rowTracking' = 'supported', 'delta.minReaderVersion' = '3', 'delta.minWriterVersion' = '7', 'delta.parquet.compression.codec' = 'zstd')"


In [0]:
# Cargar las tablas
df_region = spark.table("region")
df_cliente = spark.table("cliente")
df_consumo = spark.table("consumo_energia")

In [0]:

# Revisar esquema
df_region.printSchema()
df_cliente.printSchema()
df_consumo.printSchema()

root
 |-- id_region: string (nullable = true)
 |-- nombre_region: string (nullable = true)

root
 |-- customer_id: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- building_size_m2: integer (nullable = true)
 |-- occupants: integer (nullable = true)
 |-- id_region: string (nullable = true)

root
 |-- customer_id: string (nullable = true)
 |-- energy_cost_brl: double (nullable = true)



### Propósito:
### Validar que las tablas tienen las columnas correctas, tipos de datos esperados y nulabilidad coherente con el modelo.

**4.2 Descripción de datos: df.describe().show() y/o %sql con funciones agregadas.**

In [0]:

#PySpark
# Descripción de columnas numéricas
df_cliente.describe(["building_size_m2", "occupants"]).show()
df_consumo.describe(["energy_cost_brl"]).show()


+-------+------------------+------------------+
|summary|  building_size_m2|         occupants|
+-------+------------------+------------------+
|  count|              5000|              5000|
|   mean|           39.5762|            2.3018|
| stddev|17.516379713618157|1.0327294253642647|
|    min|                17|                 1|
|    max|                77|                 4|
+-------+------------------+------------------+

+-------+------------------+
|summary|   energy_cost_brl|
+-------+------------------+
|  count|              5000|
|   mean| 86.87402800000002|
| stddev|24.383260557486963|
|    min|             52.52|
|    max|            158.61|
+-------+------------------+



In [0]:
%sql
-- Estadísticas básicas
SELECT 
    MIN(building_size_m2) AS min_size,
    MAX(building_size_m2) AS max_size,
    AVG(building_size_m2) AS avg_size,
    COUNT(*) AS total_clientes
FROM cliente;

min_size,max_size,avg_size,total_clientes
17,77,39.5762,5000


In [0]:
%sql
SELECT 
    MIN(energy_cost_brl) AS min_cost,
    MAX(energy_cost_brl) AS max_cost,
    AVG(energy_cost_brl) AS avg_cost,
    COUNT(*) AS total_consumos
FROM consumo_energia;

min_cost,max_cost,avg_cost,total_consumos
52.52,158.61,86.87402800000001,5000


### Propósito:
### Confirmar que Spark interpreta correctamente los tipos de datos y que coinciden con el esquema SQL.

**4.3 Consultas SELECT y GROUP BY: equivalentes en Spark y SQL comparando resultados.**

In [0]:
%sql
-- Conteo por tipo de cliente
SELECT customer_type, COUNT(*) AS total
FROM cliente
GROUP BY customer_type;

customer_type,total
residential,3261
commercial,1739


In [0]:
%sql
-- Consumo promedio por región
SELECT id_region, AVG(energy_cost_brl) AS promedio_cost
FROM consumo_energia c
JOIN cliente cl ON c.customer_id = cl.customer_id
GROUP BY id_region;

id_region,promedio_cost
Midwest,86.30726640159034
Northeast,86.92740126382299
Southeast,88.09754677754694
North,87.59463768115943
South,86.7439419087137


### **PySpark**

In [0]:
# Conteo por tipo de cliente
df_cliente.groupBy("customer_type").count().show()

+-------------+-----+
|customer_type|count|
+-------------+-----+
|  residential| 3261|
|   commercial| 1739|
+-------------+-----+



In [0]:
# Consumo promedio por región
df_consumo.join(df_cliente, "customer_id") \
          .groupBy("id_region") \
          .avg("energy_cost_brl") \
          .show()

+---------+--------------------+
|id_region|avg(energy_cost_brl)|
+---------+--------------------+
|  Midwest|   86.30726640159034|
|Northeast|   86.92740126382299|
|Southeast|   88.09754677754694|
|    North|   87.59463768115943|
|    South|    86.7439419087137|
+---------+--------------------+



### Propósito:
### Comparar resultados SQL y PySpark para validar consistencia entre ambos entornos. Detectar posibles errores de join o agregación.

**4.4 Conteos y muestras: COUNT(*), LIMIT, filtros por campo.**

In [0]:
%sql
-- Total de registros de clientes

SELECT COUNT(*) AS total_clientes FROM cliente;

total_clientes
5000


In [0]:
%sql
SELECT COUNT(*) AS total_consumos FROM consumo_energia;

total_consumos
5000


In [0]:
%sql
-- Muestra de datos
SELECT * FROM cliente LIMIT 10;

customer_id,customer_type,building_size_m2,occupants,id_region
CUSTOMER_0001,residential,24,2,Northeast
CUSTOMER_0002,commercial,24,1,Midwest
CUSTOMER_0003,commercial,24,1,Southeast
CUSTOMER_0004,residential,45,4,Northeast
CUSTOMER_0005,residential,45,4,Southeast
CUSTOMER_0006,residential,52,2,North
CUSTOMER_0007,residential,17,3,Midwest
CUSTOMER_0008,commercial,45,3,Southeast
CUSTOMER_0009,residential,45,2,Northeast
CUSTOMER_0010,commercial,45,4,Midwest


In [0]:
%sql
SELECT * FROM consumo_energia LIMIT 10;

customer_id,energy_cost_brl
CUSTOMER_0001,64.51
CUSTOMER_0002,55.26
CUSTOMER_0003,74.54
CUSTOMER_0004,147.06
CUSTOMER_0005,143.06
CUSTOMER_0006,96.7
CUSTOMER_0007,52.99
CUSTOMER_0008,102.13
CUSTOMER_0009,70.03
CUSTOMER_0010,158.14


In [0]:
%sql
-- Filtrar por región
SELECT * FROM cliente WHERE id_region = 'Northeast' LIMIT 5;

customer_id,customer_type,building_size_m2,occupants,id_region
CUSTOMER_0001,residential,24,2,Northeast
CUSTOMER_0004,residential,45,4,Northeast
CUSTOMER_0009,residential,45,2,Northeast
CUSTOMER_0011,residential,52,3,Northeast
CUSTOMER_0015,residential,45,1,Northeast


In [0]:
# Conteo de registros
print("Total clientes:", df_cliente.count())

Total clientes: 5000


In [0]:
print("Total consumos:", df_consumo.count())

Total consumos: 5000


In [0]:
# Muestra de datos
df_cliente.show(10)

+-------------+-------------+----------------+---------+---------+
|  customer_id|customer_type|building_size_m2|occupants|id_region|
+-------------+-------------+----------------+---------+---------+
|CUSTOMER_0001|  residential|              24|        2|Northeast|
|CUSTOMER_0002|   commercial|              24|        1|  Midwest|
|CUSTOMER_0003|   commercial|              24|        1|Southeast|
|CUSTOMER_0004|  residential|              45|        4|Northeast|
|CUSTOMER_0005|  residential|              45|        4|Southeast|
|CUSTOMER_0006|  residential|              52|        2|    North|
|CUSTOMER_0007|  residential|              17|        3|  Midwest|
|CUSTOMER_0008|   commercial|              45|        3|Southeast|
|CUSTOMER_0009|  residential|              45|        2|Northeast|
|CUSTOMER_0010|   commercial|              45|        4|  Midwest|
+-------------+-------------+----------------+---------+---------+
only showing top 10 rows


In [0]:
df_consumo.show(10)

+-------------+---------------+
|  customer_id|energy_cost_brl|
+-------------+---------------+
|CUSTOMER_0001|          64.51|
|CUSTOMER_0002|          55.26|
|CUSTOMER_0003|          74.54|
|CUSTOMER_0004|         147.06|
|CUSTOMER_0005|         143.06|
|CUSTOMER_0006|           96.7|
|CUSTOMER_0007|          52.99|
|CUSTOMER_0008|         102.13|
|CUSTOMER_0009|          70.03|
|CUSTOMER_0010|         158.14|
+-------------+---------------+
only showing top 10 rows


In [0]:
# Filtrado por región
df_cliente.filter(df_cliente.id_region == "Northeast").show(5)

+-------------+-------------+----------------+---------+---------+
|  customer_id|customer_type|building_size_m2|occupants|id_region|
+-------------+-------------+----------------+---------+---------+
|CUSTOMER_0001|  residential|              24|        2|Northeast|
|CUSTOMER_0004|  residential|              45|        4|Northeast|
|CUSTOMER_0009|  residential|              45|        2|Northeast|
|CUSTOMER_0011|  residential|              52|        3|Northeast|
|CUSTOMER_0015|  residential|              45|        1|Northeast|
+-------------+-------------+----------------+---------+---------+
only showing top 5 rows


### Propósito:
### Validar que los datos fueron cargados correctamente, que los filtros funcionan y que podemos acceder a muestras representativas.

#Punto 5.
####Ventajas y desventajas: SQL vs Spark


##5.1 SQL

###Ventajas

1. ***Facilidad de uso**:* Su sintaxis es declarativa y rápida de aprender.

2.  ***Alta expresividad para consultas:*** Contiene filtros, agrupaciones y joins.

3. ***Integración con herramientas BI:*** Contiene integraciones con herramientas como Power BI, Tableau, Looker. 

4. ***Rendiemito optimizado:*** Su rendimiento en motores tradicionales es optimizado.

###Desventajas

1. ***Limitaciones:***  tiene limitaciones para pipelines complejos de transformación. 

2. ***Manejo de datos:*** tiene un manejo limitado de datos no estructurados. 

3. ***Escalabilidad:*** Su escalabilidad es totalmente restringida a una sola maquina o motor. 

4. ***UDFs:*** Estas son costosas y poco flexibles. 


## 5.2 Spark(PySpark)

###Ventajas

1. ***Escalabilidad:*** Cuenta con una escalabilidad bastante alta. Usa Cluster computing.

2. ***APIs:*** cuenta con apis como DataFrames, SQL, RDDs.

3. ***Capacidad:*** Es capas de procesar grandes volúmenes de datos distribuidos.

4. ***Soporte:*** Cuenta con soporte para UDFs, MLlib, streaming, gráficos. 

###Desventajas

1. ***Curva de aprendizaje:*** Cuenta con una curva de aprendizaje mayor, sobre todo cuando se trata de optimización. 

2. ***Configuración:*** Requiere configuración para sus clústers, lo que lo hace menos trivial que un motor SQL.

3. ***Costos computacionales:*** sus costos son mayores en entornos no gratuitos.

4. ***Depuración*** sus depuraciones son mucho más complejas que un SQL tradicional. 

![](SQL Vs Spark.png)